In [48]:
import torch
import torchvision.models as models

In [49]:
import os
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

In [50]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.data import DataLoader

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
## download the dataset from the following link below
#  https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset

In [52]:
class training(Dataset):
    def __init__(self,path,transform=None):
        self.path=path   
        self.transform=transform
        image=[]
        label=[]
        classes=[]
        length=len(image)
        self.calsses=classes
        for i in os.listdir(path):
            path1=os.path.join(path,i)
            classes.append(i)
            for j in os.listdir(path1):
                img_path=os.path.join(path1,j)
                image.append(img_path)
                x=classes.index(i)
                label.append(x)
        self.images=image
        self.labels=label
    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        image_path=self.images[idx]
        image=cv2.imread(image_path)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        target=self.labels[idx]
        if self.transform is not None:
            image=self.transform(image)
            target=np.eye(15)[target] ##always use one-hot encoding for crossentropy loss function
        label=torch.tensor([target])
        return image,label

In [53]:
train_transforms = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize((100, 100)),
  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [54]:
dataset=training(r"F:\datasets\Vegetable Images\train",transform=train_transforms)

In [55]:
train_set, test_set = torch.utils.data.random_split(dataset,[10000,5000])

In [56]:
train_dataloader = DataLoader(train_set, batch_size=30,shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=1,shuffle=True)

## loading the pretrained model from pytorch

In [57]:
model = models.resnet18()

## suppose if you are doing N classes classification then output features in the last linear layer must be N.

In [58]:
model.fc=Linear(in_features=512, out_features=15, bias=True)

In [59]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [60]:
model = model.to(device=device) 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001) 

## Training

In [61]:
model.train()
MAX_EPOCHS = 1
for epoch in range(MAX_EPOCHS):
    running_loss = 0.0
    for input_tensor, labels in tqdm(dataset_loader):
        images = input_tensor.to(device).float()
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels[0])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()/30 #(30=batchsize)
    print('epoch', epoch, 'loss', running_loss)

100%|████████████████████████████████████████████████████████████████████████████| 15000/15000 [45:21<00:00,  5.51it/s]

epoch 0 loss 25488.403936794857


## Testing 

In [72]:
model.eval()
ground_truth=[]
predection=[]
for img,label in tqdm(test_dataloader):
    output=model(img.to(device).float())
    target=label.to(device)
    output=torch.argmax(output).data.cpu().numpy() #the output is in tensors format.so,we are changing it to arrays to process the data in sklearn
    target=torch.argmax(target).data.cpu().numpy()
    ground_truth.append(target)
    predection.append(output)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [02:21<00:00, 35.30it/s]


## classification report

In [68]:
from sklearn.metrics import confusion_matrix,classification_report,f1_score

In [69]:
#x=confusion_matrix(df1['ground_truth'],df1['predections'])
confusion_matrix(ground_truth, predection)

array([[160,  87,   0,  15,  49,   4,   0,   0,   6,  16,   0,   0,   1,
          3,   3],
       [  1, 327,   1,  10,   0,   0,   0,   0,   0,   3,   2,   0,   0,
          0,   0],
       [  0,  29, 289,   3,   5,   4,   0,   0,   0,   0,  25,   0,   1,
          0,   0],
       [  0,  38,   2, 225,   8,   5,   0,   0,   2,   2,  11,   0,   7,
          1,   0],
       [  1,   4,   0,   4, 282,   4,   0,   0,   6,   0,   1,   0,   7,
          0,   0],
       [  0,   6,   0,  21,  25, 242,   0,   0,   7,   0,   4,   0,  39,
          0,   2],
       [  1,   3,   2,   1,   4,   1, 303,  13,   0,   0,   5,   4,   2,
          8,   3],
       [  0,   0,   0,   0,   0,   0,   0, 319,   1,   0,   0,   3,   3,
          6,   0],
       [  1,   1,   0,   2,  16,   7,   0,   0, 297,   0,   1,   5,  16,
          3,   0],
       [  3,  73,   1,  23,   1,   0,   1,   0,   1, 225,   1,   0,   3,
          1,   2],
       [  0,  22,   2,   4,   1,   1,   1,   0,   0,   2, 288,   0,   0,
       

In [70]:
print(classification_report(ground_truth,predection))

              precision    recall  f1-score   support

           0       0.95      0.47      0.62       344
           1       0.54      0.95      0.69       344
           2       0.96      0.81      0.88       356
           3       0.68      0.75      0.71       301
           4       0.67      0.91      0.77       309
           5       0.86      0.70      0.77       346
           6       0.97      0.87      0.92       350
           7       0.94      0.96      0.95       332
           8       0.83      0.85      0.84       349
           9       0.89      0.67      0.76       335
          10       0.84      0.90      0.87       321
          11       0.92      0.94      0.93       338
          12       0.73      0.82      0.77       339
          13       0.87      0.92      0.89       313
          14       0.95      0.65      0.77       323

    accuracy                           0.81      5000
   macro avg       0.84      0.81      0.81      5000
weighted avg       0.84   

In [71]:
f1_score(ground_truth,predection,average='micro')

0.8106